Goals:
This notebook attempts to set up a POC for "Infinite Memory"
1. Store
2. Retrieve

In [167]:
import time
from IPython.display import display
from loguru import logger
from openai import OpenAI
from getpass import getpass
from pydantic import BaseModel, Field


# Helper Functions

In [168]:
openai_api_key = getpass("enter_openai_api_key")

In [ ]:
from tenacity import retry, stop_after_attempt, wait_fixed
from typing import Type, Union, Any
from llama_index.core.output_parsers.utils import parse_json_markdown
import json

openai_client = OpenAI(
            api_key=openai_api_key,  
        )

def make_request(model: str, messages: list[dict[str, str]]) -> str:
    start_time = time.time()
    if model == "gpt-4o":
        response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
        )
    elif model == "o1-preview":
        response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
        )
    else:
        raise ValueError(f"Invalid model: {model}")
    return response.choices[0].message.content


@retry(
    stop=stop_after_attempt(3),
    wait=wait_fixed(2),
)
def chat_completion_request(
    messages: list[dict[str, str]],
    model: str = "gpt-4o",
    response_model: Type[BaseModel] = None,
) -> Union[str, dict[str, Any]]:
    try:
        content = make_request(model, messages)
        if response_model is not None:
            parsed_content = parse_json_markdown(content)
            try:
                return response_model(**parsed_content)
            except TypeError as e:
                error_message = {
                    "role": "user",
                    "content": f"JSON decoding error: {e}. Please adhere to the json response format that obeys the following schema: {response_model.model_json_schema()}",
                }
                messages.append(error_message)
                logger.error(
                    f"TypeError in response_model parsing: {e}. Content: {parsed_content}"
                )
                raise
        else:
            return content
    except json.JSONDecodeError as e:
        error_message = {
            "role": "user",
            "content": f"JSON decoding error: {e}. Please adhere to the json response format that obeys the following schema: {response_model.model_json_schema()}",
        }
        messages.append(error_message)
        logger.error(f"JSON decoding error: {e}. Content: {content}")
        raise
    except Exception as e:
        logger.error(f"Error while making chat completion request: {e}")
        raise


# Intialize Memory - DB and Chat Summary

In [ ]:
from typing import Optional
class CurrentKnowledge(BaseModel):
    knowledge: Optional[str] = Field(description="The current knowledge of the user", default=None)

current_knowledge = CurrentKnowledge(knowledge='')

In [ ]:
import chromadb
chroma_client = chromadb.EphemeralClient()

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings

class CustomOpenAIEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: list[str]) -> Embeddings:
        input = [text.replace("\n", " ") for text in input]
        emb_resp = openai_client.embeddings.create(input=input, model='text-embedding-3-small').data
        return [emb.embedding for emb in emb_resp]


In [ ]:
collection = chroma_client.get_or_create_collection(name="user_history", embedding_function=CustomOpenAIEmbeddingFunction()) 

# Create functions to add, delete and query DB

In [ ]:
# deleting
def delete_from_index(collection, ids):
    collection.delete(
        ids=ids
    )
delete_from_index(collection=collection,ids=['ce617a99-cc06-478c-9ddb-7f041572a139',
  'b5c1f87b-3e66-4dd0-8bcf-02b6d523a74c'])

Delete of nonexisting embedding ID: ce617a99-cc06-478c-9ddb-7f041572a139
Delete of nonexisting embedding ID: b5c1f87b-3e66-4dd0-8bcf-02b6d523a74c
Delete of nonexisting embedding ID: ce617a99-cc06-478c-9ddb-7f041572a139
Delete of nonexisting embedding ID: b5c1f87b-3e66-4dd0-8bcf-02b6d523a74c


In [ ]:
import uuid
# inserting
def insert_to_index(collection, documents, ids, metadatas=None):
    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids,
    )
    logger.info(f"Successfully inserted {len(documents)} documents")

insert_to_index(collection=collection, documents=['Fu Nan is a boy', 'Jane is a girl'])

TypeError: insert_to_index() missing 1 required positional argument: 'ids'

In [178]:
class QueryResult(BaseModel):
    ids: list[list[str]]
    documents: list[list[str]]
    distances: list[list[float]]
    metadatas: list[list[Optional[dict[str, str]]]]

# querying https://docs.trychroma.com/guides#filtering-by-metadata
def query_index(query_texts, n_results=1, where=None, where_document=None):
    query_result = collection.query(
        query_texts=query_texts,
        n_results=n_results,
        where=where,
        where_document=where_document,
    )
    return QueryResult(**query_result)
    
query_result = query_index(
    query_texts=["female", "male"]
)

In [ ]:
def update_index(ids, documents, metadatas=None):
    collection.update(
        ids=ids,
        metadatas=metadatas,
        documents=documents,
    )
    logger.info(f"Successfully updated {len(ids)} documents")

# /Store

## extract_snippets_from_conversation

In [ ]:
from typing import Optional
from datetime import datetime


class Snippet(BaseModel):
    text: str
    date_of_event: Optional[str] = Field(description="to be filled in if the snippet is an event", default=None) # TODO not sure what to do with this info for now
    id: Optional[str] = Field(description="id of the snippet", default=None)

class ConversationSnippets(BaseModel):
    snippets: list[Snippet]

extract_snippets_from_conversation_prompt = """\
You are to extract snippets of a given conversation between a career confidante and a user, which the confidante should take node of. Think of it as the confidante jotting key points down during the conversation in their journal.
Each snippet has to contain sufficient information to stand alone and be understood without the context of the entire conversation.

**
IMPORTANT: Only return the output in JSON format. The JSON structure should be a list of snippet objects, each with the fields:
	•	"text" (str): The extracted text snippet from the conversation.
	•	"date_of_event" (string): The date of the event mentioned in the snippet. If the snippet is not about an event, this field should be null. Date shouuld be formatted as "YYYY-MM-DD".

Example conversation that happend on 2024-02-01:
User: I am a software engineer and I am considering a career change.
Confidante: What are you considering?
User: I am considering becoming a data scientist.
Confidante: What is motivating you to make this change?
User: I am interested in working with data and I want to leverage my programming skills. I am also going to start taking a course in data science.
Confidante: That's awesome, when do you plan to start the course?
User: I plan to start next month.
Confidante: Great!

Example JSON:
{{
    "snippets": [
        {{
            "text": "User is considering becoming a data scientist.",
            "date_of_event": null
        }},
        {{
            "text": "User is interested in working with data and wants to leverage programming skills. User is also going to start taking a course in data science.",
            "date_of_event": null
        }},
        {{
            "text": "User plans to start data science course next month.",
            "date_of_event": "2024-03-01"
        }}

    ]
}}
===== END OF EXAMPLE ======

The 'snippets' key must be a list of snippets.
The result must be a list of objects with 'text' and 'date_of_event' keys.
Ensure each snippet contains sufficient information to stand alone and be understood without the context of the entire conversation.
**

Conversation that happened on {date}:
{conversation}

JSON:
"""

def _get_date_today():
    return datetime.now().strftime("%Y-%m-%d")


def _construct_conversation(user_messages:list[str], assistant_messages:list[str])->str:
    conversation = []
    for user_message, assistant_message in zip(user_messages, assistant_messages):
        conversation.append(f"User: {user_message}")
        conversation.append(f"Confidante: {assistant_message}")
    return "\n".join(conversation)

def extract_snippets_from_conversation(user_messages:list[str], assistant_messages:list[str]):
    conversation = _construct_conversation(user_messages, assistant_messages)
    prompt = extract_snippets_from_conversation_prompt.format(date=_get_date_today(), conversation=conversation)
    conversation_snippets: ConversationSnippets = chat_completion_request(
        messages=[
            {"role": "user", "content": prompt}
        ],
        response_model=ConversationSnippets
    )
    logger.info(f"Successfully extracted {len(conversation_snippets.snippets)} snippets from conversation")
    return conversation_snippets

In [ ]:
# test case
test_snippets = extract_snippets_from_conversation(
    user_messages=["I just got laid off from my job.", "I am considering a career change.", "First, I am thinking of starting a course in data science.", "tomorrow"],
    assistant_messages=["What are you considering?", "What are your interests?", "When do you plan to start the course?", "That's great!"]
)
test_snippets

ConversationSnippets(snippets=[Snippet(text='User just got laid off from their job.', date_of_event=None), Snippet(text='User is considering a career change.', date_of_event=None), Snippet(text='User is thinking of starting a course in data science.', date_of_event=None), Snippet(text='User plans to start the data science course tomorrow.', date_of_event='2024-11-21')])

## Insert snippets to index

In [172]:
# insert to index
def insert_snippets_to_index(collection, conversation_snippets: ConversationSnippets):
    insert_to_index(
        collection=collection, 
        documents=[snippet.text for snippet in conversation_snippets.snippets], 
        ids=[snippet.id for snippet in conversation_snippets.snippets],
        metadatas=[{"date_of_event": snippet.date_of_event} if snippet.date_of_event else {"date_of_event": ""} for snippet in conversation_snippets.snippets]
    )
        
insert_snippets_to_index(collection=collection, conversation_snippets=test_snippets)

AttributeError: 'Snippet' object has no attribute 'id'

## [to be done] Deduplicating/Updating snippets against content in index

In [ ]:
str(uuid.uuid4())

'c46f5c9b-464e-4e8f-8086-9c82b8057f98'

In [ ]:
[str(uuid.uuid4()) for i in range(len(documents))]

['962f71ce-c1b5-4542-bd0b-a209e52ad1c3']

In [170]:
from typing import Optional
from datetime import datetime


class TagSnippetsWithDbActions(BaseModel):
    snippets_to_add: Optional[list[Snippet]] = Field(description="The snippets to add to the database. Do not need id for these, as their ids will be generated upon insertion into the databse.", default=None)
    snippets_to_update: Optional[list[Snippet]]  = Field(description="The snippets to update in the database", default=None)
    snippets_to_delete: Optional[list[Snippet]]  = Field(description="The snippets to delete from the database", default=None)


TAG_PROMPT = """\
TASK
You can imagine that you are maintaining a journal of the user's career journey. 
Your task is to decide which snippets to add, update and delete in order to maintain a coherent memory the user.
You should return ids and texts of snippets to add to the database.
You are allowed to modify the text to maintain a coherent memory, but ensure the ids remain the same.
You will be shown latest conversation snippets and prior snippets that are related to the current conversation.
You are careful to insert the latest snippets while updating/deleting prior related snippets in order to maintain a coherent memory of the user's career journey.

Prior related snippets sare extracted from an existing database(journal), and should either be deleted or updated based on the latest conversation text snippets. Ensure that the ids match the ids of the snippets in the database.
Latest conversation texts are from the latest conversation between the user and their career confidant and should either be ignored or added. There is NO NEED to add the ids for them.

You are provided with snippets of the latest conversation between a user and their career confidant, and prior related snippets that are already in memory.

**
EXAMPLE_INPUT:
{{
    "latest_conversation_snippet_texts from 2024-11-20": [
        "User previously considred becoming a data scientist.",
        "User is considering becoming a softare engineer.",
        "User has tried the Data Science course, and it doesn't really interest them."
        "User got laid off from their job.",
    ]
    "prior_related_snippets_extracted_from_db": [
        {{
            "text": "User is considering becoming a data scientist.",
            "id": '644ab910-aac1-45c8-acc0-1eef35d9f4e3'
        }},
        {{
            "text": "User is interested in working with data and wants to leverage programming skills. User is also going to start taking a course in data science.",
            "id": '49fbf3e3-3e68-4b0d-9df1-747af9778e94'
        }},
        {{
            "text": User just got laid off from their job, yesterday",
            "date_of_event": "2024-11-18"
            "id": 'ce617a99-cc06-478c-9ddb-7f041572a139',
        }},
    ]
}}

EXAMPLE_OUTPUT:
{{
    "snippets_to_add": [
        {{
            "text": "User is considering becoming a softare engineer.",
        }},
    ],
    "snippets_to_update": [
        {{
            "text": "User has tried the Data Science course, and it doesn't really interest them.",
            "id": '49fbf3e3-3e68-4b0d-9df1-747af9778e94'
        }}
    ],
    "snippets_to_delete": []
}}

**

OUTPUT FORMAT:
{output_format}

INPUT:
{input}
"""

def _format_input(conversation_snippets:ConversationSnippets, prior_related_snippets:list[Snippet])->str:
    latest_conversation_snippet_texts = [snippet.text for snippet in conversation_snippets.snippets]
    # prior_related_snippets_extracted_from_db = [{"text": snippet.text, "id": snippet.id, "date_of_event": snippet.date_of_event} for snippet in prior_related_snippets.snippets]
    return str({
        "latest_conversation_snippet_texts": latest_conversation_snippet_texts,
        "prior_related_snippets_extracted_from_db": [obj.model_dump() for obj in prior_related_snippets]
    })

def _tag_db_action_to_snippet(conversation_snippets: ConversationSnippets, prior_related_snippets: list[Snippet], model: str) -> TagSnippetsWithDbActions:
    prompt = TAG_PROMPT.format(
        output_format=TagSnippetsWithDbActions.model_json_schema(),
        input=_format_input(conversation_snippets, prior_related_snippets)
    )
    tag_snippets_with_db_actions: TagSnippetsWithDbActions = chat_completion_request(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        response_model=TagSnippetsWithDbActions
    )
    logger.info(f"Successfully tagged snippets with db actions")
    return tag_snippets_with_db_actions
    


In [ ]:
tag_snippets_with_db_actions = _tag_db_action_to_snippet(
    conversation_snippets=ConversationSnippets(
        snippets=[Snippet(text="User wants to be a software_engineer.", date_of_event="2024-11-20")],
    ),
    prior_related_snippets=test_snippets.snippets,
    model="o1-preview",
)

def _retrieve_related_snippets(snippet: Snippet, n_results: int = 3) -> list[Snippet]:
    query_result = query_index(
        query_texts=[snippet.text],
        n_results=n_results,
    )
    query_result = _process_related_snippets(query_result)
    
    return query_result

def _process_related_snippets(query_result: QueryResult)->list[Snippet]:
    related_snippets = []
    for doc, id, metadata in enumerate(query_result.documents[0], query_result.ids[0], query_result.metadatas[0]):
        if query_result.metadatas[0] is not None:
            
        related_snippets.append(Snippet(text=snippet, date_of_event=query_result.metadatas[0].get("date_of_event")))
    return related_snippets


IndentationError: expected an indented block after function definition on line 15 (1125354561.py, line 17)

In [174]:
collection.query(
    query_texts=["doc10"],
    n_results=3,
)

{'ids': [['607de118-cdbb-4c5b-9dce-0d0ee0742d4f',
   'a35f24ec-3ee0-4476-927a-2da5b96966f9',
   '2742d17e-96a6-4334-b78c-42c27f6bffae']],
 'embeddings': None,
 'documents': [['User is considering a career change.',
   'User is considering a career change.',
   'User is considering a career change.']],
 'uris': None,
 'data': None,
 'metadatas': [[{'date_of_evebt': ''},
   {'date_of_evebt': ''},
   {'date_of_evebt': ''}]],
 'distances': [[1.624813437461853, 1.6248500347137451, 1.6248500347137451]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [ ]:
from typing import Optional
from datetime import datetime

class Snippet(BaseModel):
    snippet: str
    date_of_event: Optional[str] = Field(description="to be filled in if the snippet is an event", default=None)

class ConversationSnippets(BaseModel):
    snippets: list[Snippet]

extract_snippets_from_conversation_prompt = """\
You are to extract snippets of a given conversation between a career confidante and a user, which the confidante should take node of. Think of it as the confidante jotting key points down during the conversation in their journal.
Each snippet has to contain sufficient information to stand alone and be understood without the context of the entire conversation.

**
IMPORTANT: Only return the output in JSON format. The JSON structure should be a list of snippet objects, each with the fields:
	•	"snippet" (str): The extracted snippet from the conversation.
	•	"date_of_event" (string): The date of the event mentioned in the snippet. If the snippet is not about an event, this field should be null. Date shouuld be formatted as "YYYY-MM-DD".

Example conversation that happend on 2024-02-01:
User: I am a software engineer and I am considering a career change.
Confidante: What are you considering?
User: I am considering becoming a data scientist.
Confidante: What is motivating you to make this change?
User: I am interested in working with data and I want to leverage my programming skills. I am also going to start taking a course in data science.
Confidante: That's awesome, when do you plan to start the course?
User: I plan to start next month.
Confidante: Great!

Example JSON:
{{
    "snippets": [
        {{
            "snippet": "User is considering becoming a data scientist.",
            "date_of_event": null
        }},
        {{
            "snippet": "User is interested in working with data and wants to leverage programming skills. User is also going to start taking a course in data science.",
            "date_of_event": null
        }},
        {{
            "snippet": "User plans to start data science course next month.",
            "date_of_event": "2024-03-01"
        }}

    ]
}}
===== END OF EXAMPLE ======

The 'snippets' key must be a list of snippets.
The result must be a list of objects with 'snippet' and 'date_of_event' keys.
Ensure each snippet contains sufficient information to stand alone and be understood without the context of the entire conversation.
**

Conversation that happened on {date}:
{conversation}

JSON:
"""

def determine_snippets_to_add_or_delete():
    pass

In [ ]:
def delete_documents_from_index():
    pass

## Putting it all together

In [ ]:
def store(user_messages:list[str], assistant_messages:list[str]):
    conversation_snippets = extract_snippets_from_conversation(
        user_messages=user_messages,
        assistant_messages=assistant_messages
    )
    # determine_snippets_to_add_or_delete()
    # delete_documents_from_index()
    insert_snippets_to_index(collection=collection, conversation_snippets=conversation_snippets)
    logger.info(f"There are now {collection.count()} documents in the index")


In [ ]:
# Test
store(
    user_messages=["I just got laid off from my job.", "I am considering a career change.", "First, I am thinking of starting a course in data science.", "tomorrow"],
    assistant_messages=["What are you considering?", "What are your interests?", "When do you plan to start the course?", "That's great!"]
)

2024-11-20 23:22:24.764 | INFO     | __main__:extract_snippets_from_conversation:82 - Successfully extracted 4 snippets from conversation
2024-11-20 23:22:25.760 | INFO     | __main__:insert_to_index:9 - Successfully inserted 4 documents
2024-11-20 23:22:25.762 | INFO     | __main__:store:9 - There are now 20 documents in the index


# /Retrieve

In [ ]:
MIN_DISTANCE=1.3
K=10

def _build_context(query_result: QueryResult, min_distance:float)->str:
    documents = query_result.documents[0]
    distances = query_result.distances[0]
    context = ["Here are some notes from previous conversations between you and the user that might be relevant to you. Note that this snippets are from conversations that happened in the past."]
    context_num = 1
    seen_contexts = set() # to handle exact duplicates that could inadvertedly be in the index
    for document, distance in zip(documents, distances):
        if distance < min_distance and document not in seen_contexts:
            context.append(f"{context_num}: {document}")
            context_num += 1
            seen_contexts.add(document)
    return "\n".join(context)


def retrieve(content_to_retrieve:str, min_distance:float=MIN_DISTANCE, k:int=K):
    query_result = query_index(
        query_texts=[content_to_retrieve],
        n_results=k
    )
    return _build_context(query_result, min_distance)

    

In [ ]:
# test
retrieval_result = retrieve("when is the user going to start a course?")
print(retrieval_result)

Here are some notes from previous conversations between you and the user that might be relevant to you. Note that this snippets are from conversations that happened in the past.
1: User is thinking of starting a course in data science.
2: User plans to start the data science course tomorrow.
3: User is considering a career change.


# /Recap

In [ ]:
def recap():
    return current_knowledge.knowledge

UPDATE_KNOWLEDGE_PROMPT = """\
You are a career confidante. Given a conversation that just happened between you and the user, and your current knowledge of the user, update your knowledge of the user.
In your updated knowledge, you should include useful information for future interactions with the user.
The conversation just happened, so you should integrate the new information from the conversation into your updated knowledge. 

Return only the updated knowledge in JSON format.

The conversation is as follows:
{conversation}

Your current knowledge of the user is as follows:
{knowledge}

Response Format:
{{
    "knowledge": "Updated knowledge of the user."
}}
"""

def update_knowledge(user_messages:list[str], assistant_messages:list[str], current_knowledge: CurrentKnowledge):
    conversation = _construct_conversation(user_messages, assistant_messages)
    prompt = UPDATE_KNOWLEDGE_PROMPT.format(conversation=conversation, knowledge=current_knowledge)
    current_knowledge: CurrentKnowledge = chat_completion_request(
        messages=[
            {"role": "user", "content": prompt}
        ],
        response_model=CurrentKnowledge
    )
    return current_knowledge

In [ ]:
current_knowledge = update_knowledge(
    user_messages=["I just got laid off from my job.", "I am considering a career change.", "First, I am thinking of starting a course in data science.", "tomorrow"],
    assistant_messages=["What are you considering?", "What are your interests?", "When do you plan to start the course?", "That's great!"],
    current_knowledge=current_knowledge
)
print(current_knowledge)

current_knowledge = update_knowledge(
    user_messages=["I have found a job."],
    assistant_messages=["That's great!"],
    current_knowledge=current_knowledge
)
print(current_knowledge)

knowledge='The user has recently been laid off from their job and is considering a career change. They are interested in data science and plan to start a course in this field tomorrow.'
knowledge='The user has found a new job. They were recently laid off and were considering a career change, with an interest in data science. They planned to start a course in data science, but it is unclear if they have started or completed it. Future interactions should explore their new job role, satisfaction with the position, and whether they are still pursuing data science education or career change.'


- Recap function
- Prompt for context after recap/ retrieve
- Prompt for context after receive has to include sorting on memories.
- Need to include time of insertion?
- Managing memory


/retrieve
- has to include date of event if available
- [optional] probably needs to include a sorted order of ingestion time?

/store 
- needs a way to modify memories. The idea is probably to provide a list of IDs and similar documents. Then ask it what we need to combine/update. 
- returns 2 fields (to_add, to_delete). They are lists of DocumentNode objects. (thinking that update can be replaced by add and delete functions.)
- need to find a nice way to prompt 


[Optional]
- Actively pushing events.
- conversation chaining
- 